<a href="https://colab.research.google.com/github/Jarayasarabia1/Proyecto-Bootcamp-Especialidad-Ciencia-de-Datos/blob/main/Modulo9_trabajofinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
unploaded = files.upload()

Saving migraciones.csv to migraciones.csv


In [ ]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator


# 1). Carga y exploración de datos.

# Inicializar Spark
spark = SparkSession.builder.appName("migraciones.csv").getOrCreate()

# Cargar el dataset en Spark
df = spark.read.csv("migraciones.csv", header=True, inferSchema=True)

# Mostrar el dataset original
df.show()  # Muestra las primeras 20 filas

# 2). Procesamiento de datos con RDDs y DataFrames.

# Convertir el DataFrame a RDD
rdd = df.rdd

# Aplicar transformaciones sobre RDD (ejemplo con filter y map)
rdd_filtered = rdd.filter(lambda row: row['Razón'] == 'Económica')
rdd_mapped = rdd_filtered.map(lambda row: (row['Origen'], row['Destino']))

# Acciones sobre RDD
print(rdd_mapped.collect())  # Mostrar los resultados

# Realizar operaciones con DataFrames: Filtrado, Agregaciones, y Ordenamiento
df_filtered = df.filter(col("Razón") == "Económica")
df_aggregated = df_filtered.groupBy("Origen").agg({"PIB_Origen": "avg"})
df_sorted = df_aggregated.orderBy(col("avg(PIB_Origen)").desc())
df_sorted.show()

# Escribir los resultados en formato Parquet
df_sorted.write.parquet("/path/to/output.parquet", mode="overwrite")

# 3). Consultas con Spark SQL

# Registrar el DataFrame como tabla temporal
df.createOrReplaceTempView("migraciones")

# Consultar los principales países de origen y destino
result_sql = spark.sql("SELECT Origen, Destino, COUNT(*) AS Migraciones FROM migraciones GROUP BY Origen, Destino ORDER BY Migraciones DESC")
result_sql.show()

# Analizar las principales razones de migración por región
result_reasons = spark.sql("SELECT `Razón`, COUNT(*) AS Total FROM migraciones GROUP BY `Razón` ORDER BY Total DESC")
result_reasons.show()

# 4). Aplicación de MLlib para predicción de flujos migratorios

# Definir el modelo de regresión logística
lr = LogisticRegression(featuresCol="features", labelCol="Migración_Label")

# Ajustar el modelo
lr_model = lr.fit(df_ml)

# Realizar las predicciones
predictions = lr_model.transform(df_ml)

# Evaluar el modelo con BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(labelCol="Migración_Label", metricName="areaUnderROC")

# Calcular la precisión (área bajo la curva ROC)
accuracy = evaluator.evaluate(predictions)
print(f"Precisión del modelo (AUC): {accuracy}")

# Mostrar las primeras predicciones para análisis
predictions.select("ID", "Origen", "Destino", "Razón", "Migración_Label", "prediction", "probability").show(10)


+---+---------+---------------+----+---------+----------+-----------+---------------------+----------------------+----------------------+-----------------------+----------------+-----------------+
| ID|   Origen|        Destino| Año|    Razón|PIB_Origen|PIB_Destino|Tasa_Desempleo_Origen|Tasa_Desempleo_Destino|Nivel_Educativo_Origen|Nivel_Educativo_Destino|Población_Origen|Población_Destino|
+---+---------+---------------+----+---------+----------+-----------+---------------------+----------------------+----------------------+-----------------------+----------------+-----------------+
|  1|   México|           EEUU|2015|Económica|      8900|      62000|                  5.2|                   3.8|                   8.5|                   12.3|       125000000|        331000000|
|  2|    Siria|       Alemania|2016|Conflicto|      2500|      45000|                 15.4|                   4.5|                   6.2|                   14.1|        18000000|         83000000|
|  3|Venezuela|